# Restricted Boltzman Machine 

A restricted Boltzmann Machine is an "Energy Based" generative stochastic model. They were initially invented by Paul Smolensky in 1986 and were called "Harmonium". After the evolution of training algorithms in the mid 2000's by Geoffrey Hinton, the boltzman machine became more prominent. They gained big popularity in recent years in the context of the Netflix Prize where RBMs achieved state of the art performance in collaborative filtering and have beaten most of the competition.

RBM's are useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling. 


## Architecture
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. As we move forward we would learn why the output layer won't be needed.
<img src="figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [7]:
from platform import python_version
print(python_version())

3.6.8


In [8]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state

In [9]:
pip install xlrd

In [10]:
# graded
# import data

df = pd.read_excel('amazon.xlsx')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [11]:
#run this and check if you have got the correct output
df.head()

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, you find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. You train your RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, let's convert the train and test data you split above into a bag of words model.

In [13]:
#graded
# create bag of words model on train and test data

tf=CountVectorizer(stop_words='english') #the final shape should be (number of documents, vocabulary)
print(tf)
# fit tf on the dataframe df
tf = tf.fit(df['Text'])
# transform df dataframe
trainX = tf.transform(df['Text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [14]:
df['Text'].apply(lambda x: x.lower())
vectorizer = CountVectorizer()
trainX = vectorizer.fit_transform(df.iloc[:,0])
print(trainX)


  (0, 1491)	1
  (0, 1609)	1
  (0, 854)	1
  (0, 1074)	1
  (0, 1766)	1
  (0, 653)	1
  (0, 993)	1
  (0, 1640)	1
  (0, 1212)	1
  (0, 857)	1
  (0, 814)	2
  (0, 762)	1
  (0, 1604)	1
  (0, 1714)	1
  (0, 1702)	1
  (0, 711)	1
  (0, 233)	1
  (0, 367)	1
  (1, 715)	1
  (1, 259)	1
  (1, 569)	1
  (1, 1733)	1
  (2, 653)	1
  (2, 1604)	1
  (2, 721)	1
  :	:
  (997, 474)	1
  (997, 858)	1
  (997, 989)	1
  (998, 854)	1
  (998, 993)	1
  (998, 1604)	2
  (998, 1602)	1
  (998, 1109)	1
  (998, 1614)	1
  (998, 1323)	1
  (998, 453)	1
  (998, 829)	1
  (998, 1221)	1
  (998, 852)	1
  (999, 1604)	1
  (999, 1843)	1
  (999, 1081)	1
  (999, 1812)	1
  (999, 243)	1
  (999, 1065)	1
  (999, 240)	1
  (999, 1824)	1
  (999, 1105)	1
  (999, 1699)	1
  (999, 95)	1


In [15]:
#check if you are getting the correct output
print(sum(trainX.toarray()[1]))
trainX.shape

4


(1000, 1847)

You should get:
<br>

3 <br>
(1000, 1825)

Now that you have the bag of words model, let's define the number of visible and hidden units.

In [16]:
#graded
# define visible units
visibleUnits = trainX.shape[1] # vocabulary size ~1 line

# assign number of units
hiddenUnits = 5 # hyperparameter, this means that we are looking for 5 topics

In [17]:
visibleUnits

1847

In [18]:
#graded
# utility Functions
import math
# deine the sigmoid function
def sigmoid(x):
    #print(x)
    return 1 / (1 + np.exp(-x))

## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

The joint distribution is known as the Boltzmann Distribution which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and conditional probabilities of state v given the state h:
<img src="figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

Let's look at *sampleHiddenLayer* now.

### Sample Hidden Layer

You already know that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

In [19]:
#graded
def sampleHiddenLayer(v_sample):
    # write the code for calculation of hPdf using vectorized implementation of Eq 4
    
    hPdf = hiddenBias.reshape(hiddenBias.shape[0],1) + np.dot(W.transpose(),v_sample.transpose())
    #print("hPdf###: ", hPdf)
    signoid_func = np.vectorize(sigmoid)
    #print("signoid_func: ", signoid_func)
    hPdf = signoid_func(hPdf)
    #print("hPdf***: ", hPdf)
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, you are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
    

In [20]:
#graded
def sampleVisibleLayer(h_sample, D):
    # complete the following function such that vPdf has the sum of entries equal to 1 for each of the datapoints in the batch
    # you have to use axis = 1 in writing the denominator
    X = visibleBias.reshape(visibleBias.shape[0],1) + np.dot(W,h_sample)
    #print(X)
    numerator = np.exp(X)# ~1 line
    #print("numerator : ",numerator)
    denominator = numerator.sum(axis=1).reshape(numerator.shape[0],1)# ~1 line
    #print("denominator : ",denominator)
    vPdf = numerator/denominator# ~1 line
    #print("vPdf : ",vPdf)
    
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((vPdf.shape[0],batchSize))
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
    v_sample = v_sample.transpose()
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. Note that we are calculating the visible layer samples first and then using it to calculate he hidden layer sample. It'll become clear soon why we are doing so when you write the function for Contrastive Divergence.
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [21]:
#graded
def gibbs(h_sample, D):
    
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample,D)
    hPdf, h_sample = sampleHiddenLayer(v_sample)
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

You have learned that Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [22]:
#graded
def cd_k(data,k):
    
    D = data.sum(axis=1)
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data)# sample the hidden layer using the input data
    chain_start = hiddenSample_data

    for step in range(k):
        #print('step',step)
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start, D)# perform gibbs sampling using chain_start
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples, D)# perform gibbs sampling using hiddenSamples
    return hiddenPDF_data, visibleSamples, hiddenPDF


## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. **Figure out** the vectorized implementation for this.

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>
These will be helpful in writing the weight updates.

In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

In [23]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 100
eta = 0.05 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)
visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

# we shall use 10 interations of gibbs sampling
k=10

### Training

In [24]:
#graded
def train(dataX,k):
    
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs # calling the variables initiliazed at the begining
    
    mW = W # initialise momentum_weights
    mvisibleBias =  visibleBias # initialise momentum_visiblebiases
    mhiddenBias =  hiddenBias # initialise momentum_hiddenbiases
    for epoch in range(epochs):
        #print('epoch ',epoch)
        np_rng.shuffle(dataX) #shuffling the data
        
        for i in range(0, dataX.shape[0], batchSize):
            mData = dataX[i:i+batchSize,:]#select a batch of datapoints
            hiddenPDF_data, visibleSamples, hiddenPDF =  cd_k(mData,k) #perfrom Contrastive Divergence on the batch for k iterations
            
            W_diff = np.dot(mData.transpose(),hiddenPDF_data.transpose()) - np.dot(visibleSamples.transpose(),hiddenPDF.transpose())
            print("W_diff : ", W_diff)
            visible_bias_diff =  np.average(mData - visibleSamples,axis=0)
            hidden_bias_diff = np.average(hiddenPDF_data - hiddenPDF,axis=1)
            mW = mrate*mW - W_diff #write the momentum update equation for weight matrix
            mvisibleBias = mrate*mvisibleBias - visible_bias_diff #write the momentum update equation for visiblebias vector
            mhiddenBias =  mrate*mhiddenBias - hidden_bias_diff #write the momentum update equation for hiddenbias vector

            W = W + eta*mW    #weight update equation
            print("W : ", W)
            visibleBias = visibleBias + eta*mvisibleBias#visible bias update equation
            hiddenBias =  hiddenBias + eta*mhiddenBias#hidden bias update equation

## Train the Model

This will take around 10 minutes of time to run.

In [25]:
train(trainX.toarray(),k)

W_diff :  [[-4.02286712 -4.00041859 -4.00914407 -4.00198662 -3.96750222]
 [-5.56225604 -5.47649834 -5.500912   -5.51800963 -5.44621733]
 [-1.0046478  -1.01420519 -1.00584668 -0.98502332 -0.99649675]
 ...
 [ 2.97314479  3.06004547  2.99865096  2.98499226  2.94699291]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
W :  [[ 0.20597557  0.18781343  0.21514245  0.19689465  0.19098908]
 [ 0.28720622  0.2826357   0.26852123  0.27606137  0.24932335]
 [ 0.06202026  0.06087771  0.06006391  0.0285333   0.04640054]
 ...
 [-0.14622554 -0.13238971 -0.15939308 -0.14119861 -0.165826  ]
 [ 0.01304957 -0.00228866 -0.00709958  0.00976066  0.00329857]
 [ 0.00612779 -0.0028744  -0.00160117  0.02295    -0.00218668]]
W_diff :  [[-8.39609618e+00 -8.35359806e+00 -8.39755956e+00 -8.36252978e+00
  -8.32524621e+00]
 [-2.39365750e+00 -2.39153234e+00 -2.37956608e+00 -2.36501854e+00
  -2.36953925e+00]
 [-1.26249841e+01 -1.25739323e+01 -1.

W_diff :  [[-2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
  -2.00000000e+00]
 [-2.10000000e+01 -2.10000000e+01 -2.10000000e+01 -2.10000000e+01
  -2.10000000e+01]
 [-1.67913990e+01 -1.67956204e+01 -1.67963012e+01 -1.68033079e+01
  -1.67983841e+01]
 ...
 [ 2.08601629e-01  2.04380235e-01  2.03699396e-01  1.96692712e-01
   2.01616520e-01]
 [ 1.53348736e-09  1.35232322e-09  1.39495931e-09  1.44804978e-09
   1.38958186e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
W :  [[ 7.25283678e+00  7.23332398e+00  7.26073990e+00  7.23907700e+00
   7.22991240e+00]
 [ 6.14010837e+00  6.13089359e+00  6.11648505e+00  6.12361314e+00
   6.09303094e+00]
 [ 9.52777504e+00  9.52216332e+00  9.52501291e+00  9.48555471e+00
   9.50192605e+00]
 ...
 [-7.91992074e-01 -7.78334700e-01 -8.00479183e-01 -7.84921460e-01
  -8.02204792e-01]
 [ 1.32720064e-02 -2.42736363e-03 -7.36134115e-03  9.90726055e-03
   3.29251157e-03]
 [ 1.14964648e-03 -7.95759679e-03 -6.59

W_diff :  [[-1.00000002e+01 -1.00000002e+01 -1.00000002e+01 -1.00000002e+01
  -1.00000002e+01]
 [-1.00000000e+01 -1.00000000e+01 -1.00000000e+01 -1.00000000e+01
  -1.00000000e+01]
 [-4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
  -4.00000000e+00]
 ...
 [ 1.00007055e+00  1.00007043e+00  1.00006994e+00  1.00007332e+00
   1.00007260e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 5.66032531e-06  5.73882683e-06  5.56321214e-06  5.55464161e-06
   5.70169563e-06]]
W :  [[ 1.36034538e+01  1.35840019e+01  1.36113487e+01  1.35896925e+01
   1.35805340e+01]
 [ 1.31279029e+01  1.31186788e+01  1.31042698e+01  1.31113971e+01
   1.30808071e+01]
 [ 1.71956390e+01  1.71903551e+01  1.71933796e+01  1.71549633e+01
   1.71705400e+01]
 ...
 [-1.33155118e+00 -1.31660317e+00 -1.33762305e+00 -1.32177412e+00
  -1.33816077e+00]
 [ 1.32724408e-02 -2.42763460e-03 -7.36185246e-03  9.90754681e-03
   3.29249967e-03]
 [ 9.60143983e-04 -8.14640378e-03 -6.78

W_diff :  [[-9.00000000e+00 -9.00000000e+00 -9.00000000e+00 -9.00000000e+00
  -9.00000000e+00]
 [-9.00000000e+00 -9.00000000e+00 -9.00000000e+00 -9.00000000e+00
  -9.00000000e+00]
 [-4.99867375e+00 -4.99868428e+00 -4.99869012e+00 -4.99873981e+00
  -4.99871345e+00]
 ...
 [ 1.32625970e-03  1.31573319e-03  1.30989040e-03  1.26019693e-03
   1.28656168e-03]
 [ 8.87163111e-29  8.22462528e-29  8.18692263e-29  8.53834110e-29
   8.28711898e-29]
 [ 3.25498065e-09  3.27597993e-09  3.18191663e-09  3.17827802e-09
   3.25057406e-09]]
W :  [[ 2.38849997e+01  2.38655480e+01  2.38928946e+01  2.38712384e+01
   2.38620799e+01]
 [ 2.02821760e+01  2.02729518e+01  2.02585428e+01  2.02656702e+01
   2.02350801e+01]
 [ 2.52983109e+01  2.52930284e+01  2.52960681e+01  2.52577147e+01
   2.52732528e+01]
 ...
 [-2.19587483e+00 -2.18116796e+00 -2.20178088e+00 -2.18585024e+00
  -2.20242514e+00]
 [ 1.32724416e-02 -2.42763513e-03 -7.36185346e-03  9.90754737e-03
   3.29249965e-03]
 [ 9.59199609e-04 -8.14735463e-03 -6.78

W_diff :  [[-10.         -10.         -10.         -10.         -10.        ]
 [ -6.          -6.          -6.          -6.          -6.        ]
 [ -9.          -9.          -9.          -9.          -9.        ]
 ...
 [  0.02118729   0.02117017   0.02094182   0.02141378   0.02107288]
 [  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.        ]]
W :  [[ 3.17787137e+01  3.17592620e+01  3.17866086e+01  3.17649524e+01
   3.17557939e+01]
 [ 2.90025945e+01  2.89933703e+01  2.89789613e+01  2.89860887e+01
   2.89554986e+01]
 [ 3.40602975e+01  3.40550180e+01  3.40580588e+01  3.40197177e+01
   3.40352492e+01]
 ...
 [-2.85015945e+00 -2.83533729e+00 -2.85579545e+00 -2.83998466e+00
  -2.85633581e+00]
 [ 1.32724416e-02 -2.42763513e-03 -7.36185347e-03  9.90754737e-03
   3.29249964e-03]
 [ 9.59197293e-04 -8.14735696e-03 -6.78142591e-03  1.83161923e-02
  -7.40270968e-03]]
W_diff :  [[-3.00000000e+00 -3.00000000e+00 -3.00000000

...

W_diff :  [[-1.10000000e+01 -1.10000000e+01 -1.10000000e+01 -1.10000000e+01
  -1.10000000e+01]
 [-1.00000000e+01 -1.00000000e+01 -1.00000000e+01 -1.00000000e+01
  -1.00000000e+01]
 [-2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
  -2.00000000e+00]
 ...
 [ 4.70256830e-06  4.62895345e-06  4.70729943e-06  4.74810957e-06
   4.59046259e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.21419220e-41  2.20726196e-41  2.15207277e-41  2.17258469e-41
   2.21581872e-41]]
W :  [[ 1.24444393e+02  1.24424941e+02  1.24452288e+02  1.24430632e+02
   1.24421473e+02]
 [ 1.17639026e+02  1.17629801e+02  1.17615392e+02  1.17622520e+02
   1.17591930e+02]
 [ 1.30626827e+02  1.30621552e+02  1.30624593e+02  1.30586267e+02
   1.30601791e+02]
 ...
 [-1.02278700e+01 -1.02129820e+01 -1.02332060e+01 -1.02175087e+01
  -1.02340281e+01]
 [ 1.32724416e-02 -2.42763513e-03 -7.36185347e-03  9.90754737e-03
   3.29249964e-03]
 [ 9.59197288e-04 -8.14735697e-03 -6.78

C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


W_diff :  [[-7.00000000e+00 -7.00000000e+00 -7.00000000e+00 -7.00000000e+00
  -7.00000000e+00]
 [-1.50000000e+01 -1.50000000e+01 -1.50000000e+01 -1.50000000e+01
  -1.50000000e+01]
 [-6.00000000e+00 -6.00000000e+00 -6.00000000e+00 -6.00000000e+00
  -6.00000000e+00]
 ...
 [ 4.91258052e-06  4.84838955e-06  4.81629391e-06  4.93553336e-06
   4.82732411e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
W :  [[ 1.26873304e+02  1.26853853e+02  1.26881199e+02  1.26859543e+02
   1.26850385e+02]
 [ 1.19411879e+02  1.19402655e+02  1.19388246e+02  1.19395373e+02
   1.19364783e+02]
 [ 1.32804419e+02  1.32799144e+02  1.32802186e+02  1.32763859e+02
   1.32779383e+02]
 ...
 [-1.02590429e+01 -1.02441542e+01 -1.02643777e+01 -1.02486824e+01
  -1.02651990e+01]
 [ 1.32724416e-02 -2.42763513e-03 -7.36185347e-03  9.90754737e-03
   3.29249964e-03]
 [ 9.59197288e-04 -8.14735697e-03 -6.78

C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater_equal


ValueError: pvals < 0, pvals > 1 or pvals contains NaNs

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. You can take a look at the words under each topic and see what they are talking about. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. You can see that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary
    
Feel free to change the number of iterations of gibbs sampling in Contrastive Divergence and see how the distribution of words change under the topics.

In [ ]:
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias)
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten()
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            lm+=1
            if lm==15:
                break
        print('\n')

In [ ]:
worddist( hiddenUnits, vectorizer.vocabulary_)

The output shows the probability assigned to each word for ever topic present.